In [ ]:
import pandas
import numpy
data = pandas.read_csv('gym_MachiKoro/envs/base/card.csv', index_col='name')
data

In [ ]:
data[(data['activated_from'] == 'Anyone') | (data['activated_from'] == 'You')]

In [ ]:
from copy import deepcopy


class Card:
    def __init__(self, card_type: str, name: str, price: int, description: str):
        self.__card_type = card_type # Cái biểu tượng bên cạnh tên thẻ
        self.__name = name
        self.__price = price # Giá để mua thẻ
        self.__description = description

    @property
    def card_type(self):
        return self.__card_type
    
    @property
    def name(self):
        return self.__name

    @property
    def price(self):
        return self.__price

    @property
    def description(self):
        return self.__description

    def convert_to_dict(self):
        return deepcopy({
            'card_type': self.__card_type,
            'name': self.__name,
            'price': self.__price
        })
        

class Important_Land_Card(Card):
    def __init__(self, card_type: str, name: str, price: int, description: str):
        super().__init__(card_type, name, price, description)

class Support_Card(Card):
    def __init__(self, card_type: str, name: str, price: int, description: str, value_to_activate: list, activate_from: str,
                income: None or int, income_from: str, income_times: str, card_type_in_effect: None or str):
        super().__init__(card_type, name, price, description)
        self.__value_to_activate = value_to_activate.copy()
        self.__activate_from = activate_from
        self.__income = income
        self.__income_from = income_from
        self.__income_times = income_times
        self.__card_type_in_effect = card_type_in_effect
    
    @property
    def value_to_activate(self):
        return self.__value_to_activate.copy()
    
    @property
    def activate_from(self):
        return self.__activate_from
    
    @property
    def income(self):
        return self.__income
    
    @property
    def income_from(self):
        return self.__income_from
    
    @property
    def income_times(self):
        return self.__income_times
    
    @property
    def card_type_in_effect(self):
        return self.__card_type_in_effect

    def convert_to_dict(self):
        return deepcopy({
            'card_type': self.card_type,
            'name': self.name,
            'price': self.price,
            'value_to_activate': self.__value_to_activate.copy(),
            'activate_from': self.__activate_from,
            'income': self.__income,
            'income_from': self.__income_from,
            'income_times': self.__income_times,
            'card_type_in_effect': self.__card_type_in_effect
        })

In [ ]:
list_Important_Land_Card = []
for name in data.index[:4]:
    list_Important_Land_Card.append(Important_Land_Card(data['card_type'][name], name, int(data['price'][name]), data['description'][name]))

In [ ]:
list_Support_Card = []
for name in data.index[4:]:
    try:
        income = int(data['income'][name])
    except:
        income = None
    list_Support_Card.append(Support_Card(data['card_type'][name], name, int(data['price'][name]), data['description'][name], [int(a) for a in data['value_to_activate'][name].split(',')], data['activate_from'][name], income, data['income_from'][name], data['income_times'][name], data['card_type_in_effect'][name]))

In [ ]:
for card in list_Important_Land_Card:
    print('###', card.card_type, '***card_type')
    print(card.name, '***name')
    print(card.price, '***price')
    print(card.description, '***description')

In [ ]:
for card in list_Support_Card:
    print('###', card.card_type, '***card_type')
    print(card.name, '***name')
    print(card.price, '***price')
    print(card.description, '***description')
    print(card.value_to_activate, '***value_to_activate')
    print(card.activate_from, '***activate_from')
    print(card.income, '***income')
    print(card.income_from, '***income_from')
    print(card.income_times, '***income_times')
    print(card.card_type_in_effect, '***card_type_in_effect')

In [ ]:
temp = [card.name for card in list_Important_Land_Card]
print(temp)

In [ ]:
from copy import deepcopy
import pandas
# from gym_MachiKoro.envs.base.card import Support_Card, Important_Land_Card


class Board:
    def __init__(self):
        temp = ['Wheat Field', 'Livestock Farm', 'Bakery', 'Cafe', 'Convenience Store', 'Forest', 'Stadium', 'TV Station', 'Business Complex',
                'Cheese Factory', 'Furniture Factory', 'Mine', 'Family Restaurant', 'Apple Orchard', 'Vegetable Market']

        self.__support_cards = {}
        for name in temp:
            if name not in ['Stadium', 'TV Station', 'Business Complex']:
                self.__support_cards[name] = 6
            else:
                self.__support_cards[name] = 4
        
        data = pandas.read_csv('gym_MachiKoro/envs/base/card.csv', index_col='name')

        self.__support_cards_object = {}
        for name in temp:
            try:
                income = int(data['income'][name])
            except:
                income = None

            self.__support_cards_object[name] = Support_Card(data['card_type'][name], name, int(data['price'][name]), data['description'][name],[int(a) for a in data['value_to_activate'][name].split(',')],
                                                            data['activate_from'][name], income, data['income_from'][name], data['income_times'][name], data['card_type_in_effect'][name])

    @property
    def support_cards(self):
        return deepcopy(self.__support_cards)

    @property
    def support_cards_object(self):
        return deepcopy(self.__support_cards_object)

In [ ]:
board = Board()

In [ ]:
for key in board.support_cards_object.keys():
    print(type(board.support_cards_object[key]), board.support_cards_object[key].convert_to_dict())

In [ ]:
print(board.support_cards)

In [ ]:
from copy import deepcopy
import pandas
# from gym_MachiKoro.envs.base.card import Support_Card, Important_Land_Card


class Player:
    def __init__(self, name):
        self.__name = name
        self.__full_action = []
        self.__amount_action_space = 0
        self.reset()

    def reset(self):
        data = pandas.read_csv('gym_MachiKoro/envs/base/card.csv', index_col='name')
        temp = ['Wheat Field', 'Livestock Farm', 'Bakery', 'Cafe', 'Convenience Store', 'Forest', 'Stadium', 'TV Station', 'Business Complex',
                'Cheese Factory', 'Furniture Factory', 'Mine', 'Family Restaurant', 'Apple Orchard', 'Vegetable Market']

        self.__support_cards = {}
        for name in temp:
            self.__support_cards[name] = 0

        self.__support_cards_object = {}
        for name in temp:
            try:
                income = int(data['income'][name])
            except:
                income = None

            self.__support_cards_object[name] = Support_Card(data['card_type'][name], name, int(data['price'][name]), data['description'][name],[int(a) for a in data['value_to_activate'][name].split(',')],
                                                            data['activate_from'][name], income, data['income_from'][name], data['income_times'][name], data['card_type_in_effect'][name])

        temp = ['Train Station', 'Shopping Mall', 'Amusement Park', 'Radio Tower']
        self.__important_land_cards = {}
        for name in temp:
            self.__important_land_cards[name] = 0
        
        self.__important_land_cards_object = {}
        for name in temp:
            self.__important_land_cards_object[name] = Important_Land_Card(data['card_type'][name], name, int(data['price'][name]), data['description'][name])

    @property
    def name(self):
        return self.__name
    
    @property
    def support_cards(self):
        return deepcopy(self.__support_cards)

    @property
    def support_cards_object(self):
        return deepcopy(self.__support_cards_object)

    @property
    def important_land_cards(self):
        return deepcopy(self.__important_land_cards)
    
    @property
    def important_land_cards_object(self):
        return deepcopy(self.__important_land_cards_object)

    @property
    def amount_action_space(self):
        return self.__amount_action_space

In [ ]:
player = Player('An')

In [ ]:
for key in player.important_land_cards_object.keys():
    print(type(player.important_land_cards_object[key]), player.important_land_cards_object[key].convert_to_dict())

In [ ]:
print(player.support_cards)

In [ ]:
import random
def roll_the_dice(number_of_dice=1):
    if number_of_dice == 1:
        return random.randint(1,6)
        
    elif number_of_dice == 2:
        return random.randint(1,6), random.randint(1,6)

In [ ]:
from collections import Counter
a = Counter([roll_the_dice(2) for i in range(1000000)])
a.__len__()

In [ ]:
temp = ['A', 'B']
turn = 0
for i in range(turn-1,turn-temp.__len__(),-1):
    print(temp[i])

In [ ]:
a = {
    'a': 3,
    'b': 4,
    'c': 5,
    'd': 6
}

b = {item for item in a.items() if item[1] % 2 == 1}
b

In [ ]:
class A:
    def __init__(self):
        self.__abc = random.randint(1,5)

class B:
    def __init__(self):
        self.__list_abc = [A(), A(), A()]
        self.__choice = random.choice(self.__list_abc)

In [ ]:
import random
temp = B()

In [ ]:
temp._B__choice._A__abc = 99

In [ ]:
list_ = ['a', 'aa', 'b', 'c']
a = list_.pop(0)
list_

In [ ]:
a = list_.pop(0)
list_

In [ ]:
a = 1

In [ ]:
def a():
    return 1,2

In [9]:
b = 1

In [1]:
class A:
    def __init__(self):
        self.abc = [1,2,3]
        self.xyz()

    def xyz(self):
        self.xyz = self.abc.pop(0)

In [2]:
abcxyz = A()

In [10]:
if type(b) == tuple and b[0] == b[1]:
    print('Yes')
else:
    print('No')

No
